In [2]:
#lbbVIXAnalysis-4
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

import datetime as dt
import os
os.getcwd()

'C:\\Users\\cinderella\\dataSci\\DataScience\\eliteDataScience\\07-CapstoneProject\\Final\\_projectNotebooks'

In [3]:
os.chdir('c:\\users\\cinderella\\datasci\\datascience\\elitedatascience\\07-CapstoneProject\\Final')

From VXAnalysis-03 we've added all the VIX futures month terms (t01 - t12) along with the futures contract month columns inside them (F - Z) represented by the futures numeric month. 

Also included, a split by futures name, alpha, year and future month name (because futures contracts typically straddle months, we cannot just go by calendar year and month.. i.e.: 

> Futures Contract VX20F  <br>
> === actually was initially traded 9 months before at around April 27, 2019 <br>
> === contract ends Jan 22, 2020 <br>
> === began trading as frontMonth on Dec 18, 2019<br>

So January 2020 VX futures contract:<br>
    began: 4/27/2019<br>
    ends:  1/22/2020<br>
    t01:   12/18/2020 (when it became the "front" month contract)<br>
    LTD:   12/18/2020 8:00 AM CT <br>
    Futures month number: 1<br><br>


Now we need *only* create a way for each row's future listing by tradeDate to include its term position.
The variables numMthT10, 11 and 12, are "non month term positions". That means months listed there are not in the term structure since the term structure is only 9 months long (and in earlier years, only 7 months long)<br><br>

We are trying to match a closing price for t01, t02, ... t09 for each futures contract depending upon where they are in the cycle.



In [4]:
df04b = pd.read_csv('abt-3.csv', index_col=0)

I know of no other way than to reaffirm the datetime variables, 
Then reestablish the diffDates var so that we can create a bool (diffDates00) to compress the data with
So we will load all the datetime stuff, run the diffDates metrics and then save as csv..then bring it back in
before we do the list function

In [5]:
df04b.dtypes

Trade Date        object
Futures           object
Open             float64
High             float64
Low              float64
Close            float64
Settle           float64
Change           float64
Total Volume       int64
EFP                int64
Open Interest      int64
calYear            int64
calMth             int64
opYear             int64
opMth              int64
opFOM             object
opDOW1st           int64
opDOW2nd           int64
noYear             int64
spxDaysNum         int64
opXP              object
opXP_DOW           int64
weekday           object
t01XP             object
LTD               object
dt01              object
dt02              object
LTD-TIME          object
diffDates         object
numMthT01          int64
numMthT02          int64
numMthT03          int64
numMthT04          int64
numMthT05          int64
numMthT06          int64
numMthT07          int64
numMthT08          int64
numMthT09          int64
numMthT10          int64
numMthT11          int64


In [6]:
df04b['Trade Date'] = pd.to_datetime(df04b['Trade Date'])
df04b['opFOM'] = pd.to_datetime(df04b['opFOM'])
df04b['t01XP'] = pd.to_datetime(df04b['t01XP'])
df04b['LTD'] = pd.to_datetime(df04b['LTD'])
df04b['dt01'] = pd.to_datetime(df04b['dt01'])
df04b['dt02'] = pd.to_datetime(df04b['dt02'])
df04b['opXP'] = pd.to_datetime(df04b['opXP'])
df04b['9MthsStart'] = pd.to_datetime(df04b['9MthsStart'])
df04b['7MthsStart'] = pd.to_datetime(df04b['7MthsStart'])

In [7]:
df04b['diffDates'] = df04b['LTD'] - df04b['Trade Date']
df04b['numMthT01'] = np.where(df04b['diffDates'].dt.days >= 0, df04b['calMth'], df04b['opMth'])
df04b['diffDates00'] = (df04b['diffDates'] == '0')

This database has the new column diffDates00 acting as a mask, True for those rows with diffDates at 0 and False for all others. All rows are included in this df

In [8]:
df04b.tail(10)

,Trade Date,Futures,Open,High,Low,Close,Settle,Change,Total Volume,EFP,Open Interest,calYear,calMth,opYear,opMth,opFOM,opDOW1st,opDOW2nd,noYear,spxDaysNum,opXP,opXP_DOW,weekday,t01XP,LTD,dt01,dt02,LTD-TIME,diffDates,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,9MthsStart,7MthsStart,futAlpha,futMthName,futYear,futMth,diffDates00
33791,2020-08-10,Z (Dec 2020),27.92,28.08,27.45,27.64,27.625,-0.350,1608,0,15197,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT,9 days,8,9,10,11,12,1,2,3,4,5,6,7,2019-11-23,2020-01-22,Z,Dec,2020,12,False
33792,2020-08-11,Z (Dec 2020),27.65,28.33,27.05,28.05,28.225,0.600,4393,0,15905,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT,8 days,8,9,10,11,12,1,2,3,4,5,6,7,2019-11-23,2020-01-22,Z,Dec,2020,12,False
33793,2020-08-12,Z (Dec 2020),28.00,28.25,27.35,27.68,27.675,-0.550,3618,0,16430,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT,7 days,8,9,10,11,12,1,2,3,4,5,6,7,2019-11-23,2020-01-22,Z,Dec,2020,12,False
33794,2020-08-13,Z (Dec 2020),27.70,28.01,27.40,27.80,27.825,0.150,3136,0,16737,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT,6 days,8,9,10,11,12,1,2,3,4,5,6,7,2019-11-23,2020-01-22,Z,Dec,2020,12,False
33795,2020-08-14,Z (Dec 2020),27.85,28.37,27.77,28.25,28.300,0.475,8878,0,18722,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT,5 days,8,9,10,11,12,1,2,3,4,5,6,7,2019-11-23,2020-01-22,Z,Dec,2020,12,False
33796,2020-08-17,Z (Dec 2020),28.06,28.10,27.31,27.60,27.375,-0.925,7541,0,18656,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT,2 days,8,9,10,11,12,1,2,3,4,5,6,7,2019-11-23,2020-01-22,Z,Dec,2020,12,False
33797,2020-08-18,Z (Dec 2020),27.40,27.75,27.20,27.42,27.475,0.100,6174,0,19197,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT,1 days,8,9,10,11,12,1,2,3,4,5,6,7,2019-11-23,2020-01-22,Z,Dec,2020,12,False
33798,2020-08-19,Z (Dec 2020),27.40,28.00,27.35,27.92,27.900,0.425,6049,0,20106,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT,0 days,8,9,10,11,12,1,2,3,4,5,6,7,2019-11-23,2020-01-22,Z,Dec,2020,12,True
33799,2020-08-20,Z (Dec 2020),27.92,28.52,27.65,27.80,27.825,-0.075,6803,0,20760,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT,-1 days,9,10,11,12,1,2,3,4,5,6,7,8,2019-11-23,2020-01-22,Z,Dec,2020,12,False
33800,2020-08-21,Z (Dec 2020),27.77,28.45,27.66,28.24,28.175,0.350,5035,0,20592,2020,8,2020,9,2020-09-01,1,3,0,17,2020-09-18,4,Friday,2020-08-19,2020-08-19,2014-10-22,2015-10-21,8:00 AM CT,-2 days,9,10,11,12,1,2,3,4,5,6,7,8,2019-11-23,2020-01-22,Z,Dec,2020,12,False


In [9]:
df04b.shape 

(33792, 48)

We are going to copy this dataframe because even though we just imported from the csv, I'm so concerned that somewhere along the line there will be a linked something or other.. so from now on the df gets copied and takes on the name of the -04B suffix of the notebook it lives in.

In [10]:
df = df04b.copy()

In [11]:
df.diffDates00.value_counts()

False    32140
True      1652
Name: diffDates00, dtype: int64

In this next grouping we will create a df that only includes those columns we will need to link the futures by trade date to their term position. The database still includes all rows

In [12]:
terms = ['numMthT01', 
               'numMthT02',
               'numMthT03',
               'numMthT04',
               'numMthT05',
               'numMthT06',
               'numMthT07',
               'numMthT08',
               'numMthT09',
               'numMthT10',
               'numMthT11',
               'numMthT12']

In [13]:
##We are creating a new df to keep our slice of df in 
df04bT = df.groupby(['Trade Date','diffDates','Futures','futYear','futMth','calYear','LTD','t01XP','9MthsStart','7MthsStart','Open','High','Low','Close','Settle'], as_index=False)[terms].mean()

In [14]:
df04bT.shape

(33792, 27)

In [15]:
#save dft as a csv in to come back to after we do our lookup
df04bT.to_csv('df-4T-00.csv')

In [16]:
df = df04bT.copy()

In [17]:
df.shape

(33792, 27)

In [18]:
df.dtypes

Trade Date     datetime64[ns]
diffDates     timedelta64[ns]
Futures                object
futYear                 int64
futMth                  int64
calYear                 int64
LTD            datetime64[ns]
t01XP          datetime64[ns]
9MthsStart     datetime64[ns]
7MthsStart     datetime64[ns]
Open                  float64
High                  float64
Low                   float64
Close                 float64
Settle                float64
numMthT01               int64
numMthT02               int64
numMthT03               int64
numMthT04               int64
numMthT05               int64
numMthT06               int64
numMthT07               int64
numMthT08               int64
numMthT09               int64
numMthT10               int64
numMthT11               int64
numMthT12               int64
dtype: object

In [19]:
df.shape

(33792, 27)

In [20]:
df.sort_values(['Trade Date'],inplace=True)

In [21]:
df['termA'] = np.where(df['numMthT01'] == df['futMth'], 1,"")

In [22]:
df['termB'] = np.where((df.numMthT01 == 1)& (df.futMth != 1), df.futMth, "")

In [23]:
df['termC'] = np.where((df.numMthT01 == 2) & (df.futMth != 2), df.futMth - 1, "")

In [24]:
df['termD'] = np.where((df.numMthT01 == 3) & (df.futMth != 3), df.futMth - 2, "")

In [25]:
df['termE'] = np.where((df.numMthT01 == 4) & (df.futMth != 4), df.futMth - 3, "")

In [26]:
df['termF'] = np.where((df.numMthT01 == 5) & (df.futMth > 5),df.futMth - 4, np.where((df.numMthT01 == 5) & (df.futMth < 5),df.futMth + 8,""))

In [27]:
df['termG'] = np.where((df.numMthT01 == 6) & (df.futMth > 6),df.futMth - 5, np.where((df.numMthT01 == 6) & (df.futMth < 6),df.futMth + 7,""))

In [28]:
df['termH'] = np.where((df.numMthT01 == 7) & (df.futMth > 7),df.futMth - 6, np.where((df.numMthT01 == 7) & (df.futMth < 7),df.futMth + 6,""))

In [29]:
df['termI'] = np.where((df.numMthT01 == 8) & (df.futMth > 8),df.futMth - 7, np.where((df.numMthT01 == 8) & (df.futMth < 8),df.futMth + 5,""))

In [30]:
df['termJ'] = np.where((df.numMthT01 == 9) & (df.futMth > 9),df.futMth - 8, np.where((df.numMthT01 == 9) & (df.futMth < 9),df.futMth + 4,""))

In [31]:
df['termK'] = np.where((df.numMthT01 == 10) & (df.futMth > 10),df.futMth - 9, np.where((df.numMthT01 == 10) & (df.futMth < 10),df.futMth + 3,""))

In [32]:
df['termL'] = np.where((df.numMthT01 == 11) & (df.futMth > 11),df.futMth - 10, np.where((df.numMthT01 == 11) & (df.futMth < 11),df.futMth + 2,""))

In [33]:
df['termM'] = np.where((df.numMthT01 == 12) & (df.futMth > 12),df.futMth - 11, np.where((df.numMthT01 == 12) & (df.futMth < 12),df.futMth + 1,""))

In [34]:
df['termCount']= df.termA + df.termB + df.termC + df.termD + df.termE +df.termF + df.termG + df.termH + df.termI + df.termJ + df.termK + df.termL + df.termM  

In [35]:
df.shape

(33792, 41)

In [36]:
df.dtypes

Trade Date     datetime64[ns]
diffDates     timedelta64[ns]
Futures                object
futYear                 int64
futMth                  int64
calYear                 int64
LTD            datetime64[ns]
t01XP          datetime64[ns]
9MthsStart     datetime64[ns]
7MthsStart     datetime64[ns]
Open                  float64
High                  float64
Low                   float64
Close                 float64
Settle                float64
numMthT01               int64
numMthT02               int64
numMthT03               int64
numMthT04               int64
numMthT05               int64
numMthT06               int64
numMthT07               int64
numMthT08               int64
numMthT09               int64
numMthT10               int64
numMthT11               int64
numMthT12               int64
termA                  object
termB                  object
termC                  object
termD                  object
termE                  object
termF                  object
termG     

In [37]:
df.head(10)

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount
0,2004-03-26,-10 days,K (May 2004),2004,5,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.24,21.25,20.27,20.27,20.32,4,5,6,7,8,9,10,11,12,1,2,3,,,,,2,,,,,,,,,2
1,2004-03-26,-10 days,M (Jun 2004),2004,6,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.33,20.37,20.10,20.20,20.16,4,5,6,7,8,9,10,11,12,1,2,3,,,,,3,,,,,,,,,3
2,2004-03-26,-10 days,Q (Aug 2004),2004,8,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.04,20.13,20.00,20.06,20.11,4,5,6,7,8,9,10,11,12,1,2,3,,,,,5,,,,,,,,,5
3,2004-03-26,-10 days,X (Nov 2004),2004,11,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.17,21.37,20.88,20.89,20.94,4,5,6,7,8,9,10,11,12,1,2,3,,,,,8,,,,,,,,,8
4,2004-03-29,-13 days,K (May 2004),2004,5,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.90,19.99,19.75,19.77,19.80,4,5,6,7,8,9,10,11,12,1,2,3,,,,,2,,,,,,,,,2
5,2004-03-29,-13 days,M (Jun 2004),2004,6,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.85,19.85,19.73,19.76,19.77,4,5,6,7,8,9,10,11,12,1,2,3,,,,,3,,,,,,,,,3
6,2004-03-29,-13 days,Q (Aug 2004),2004,8,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.76,19.76,19.76,19.76,19.68,4,5,6,7,8,9,10,11,12,1,2,3,,,,,5,,,,,,,,,5
7,2004-03-29,-13 days,X (Nov 2004),2004,11,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,20.83,20.97,20.83,20.97,20.98,4,5,6,7,8,9,10,11,12,1,2,3,,,,,8,,,,,,,,,8
11,2004-03-30,-14 days,X (Nov 2004),2004,11,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,21.06,21.06,21.00,21.00,21.02,4,5,6,7,8,9,10,11,12,1,2,3,,,,,8,,,,,,,,,8
10,2004-03-30,-14 days,Q (Aug 2004),2004,8,2004,2004-03-16,2004-03-17,2003-06-21,2003-08-20,19.72,19.97,19.72,19.92,19.91,4,5,6,7,8,9,10,11,12,1,2,3,,,,,5,,,,,,,,,5


Ho hum, accuracy. As a last quality check, and now on the huge database, we have a bit of a snag. I queried the termCount (a proxy for our term position) and there are a large amount of blank sums. Those with diffDates of -1 means that the contract has changed over, and although trading may have stopped, there may still be settlement amounts that are recorded on those days. Historically, the last trading date (LTD) and settlement date were different, currently they are the same, meaning the exiting month stops at 8:30 am on the settlement date where the at the same time the frontmonth begins its "front month" status.<br>

So we will be keeping any rows that have data in them since, and quite naturally, the term position does not exist--that is why most of them do not have a term position..so let's give them a term position of 0 (not blank like the other rows unused..but zero..... but wait something doesn't make sense<br>

Aha!! another error in the formulas (only november, september , october) sheesh..(which has since been fixed) We will call these queries Quality Checks...<br>

## Quality Checks -- we need them - and why don't they work sometimes???


Yikes, did you know that the term variables we were creating out of int vars, came out as objects.. so no matter what we queried on number wise..it politely ignored us? Depending upon the way I hold my mouth and the wind velocity..sometimes term columns were typed as objects and sometimes as floats. Regardless, when they were objects and I asked whether there were negative numbers, I received no rows -- and thought erroneously that I didn't have any negative numbers, when in fact I had plenty of negative objects! grrrr

In [38]:
df.termCount.value_counts()

2     4034
1     4033
3     3801
5     3723
4     3707
6     3707
7     3524
8     3402
9     2698
10     530
12     273
11     218
0       80
-1      57
-2       5
Name: termCount, dtype: int64

In [39]:
df['termCount'] = df.termCount.apply(pd.to_numeric, errors = 'coerce')

In [40]:
df.termA = df.termA.apply(pd.to_numeric, errors = 'coerce')
df.termB = df.termB.apply(pd.to_numeric, errors = 'coerce')
df.termC = df.termC.apply(pd.to_numeric, errors = 'coerce')
df.termD = df.termD.apply(pd.to_numeric, errors = 'coerce')
df.termE = df.termE.apply(pd.to_numeric, errors = 'coerce')
df.termF = df.termF.apply(pd.to_numeric, errors = 'coerce')
df.termG = df.termG.apply(pd.to_numeric, errors = 'coerce')
df.termH = df.termH.apply(pd.to_numeric, errors = 'coerce')
df.termI = df.termI.apply(pd.to_numeric, errors = 'coerce')
df.termJ = df.termJ.apply(pd.to_numeric, errors = 'coerce')
df.termK = df.termK.apply(pd.to_numeric, errors = 'coerce')
df.termL = df.termL.apply(pd.to_numeric, errors = 'coerce')
df.termM = df.termM.apply(pd.to_numeric, errors = 'coerce')

In [41]:
df.dtypes

Trade Date     datetime64[ns]
diffDates     timedelta64[ns]
Futures                object
futYear                 int64
futMth                  int64
calYear                 int64
LTD            datetime64[ns]
t01XP          datetime64[ns]
9MthsStart     datetime64[ns]
7MthsStart     datetime64[ns]
Open                  float64
High                  float64
Low                   float64
Close                 float64
Settle                float64
numMthT01               int64
numMthT02               int64
numMthT03               int64
numMthT04               int64
numMthT05               int64
numMthT06               int64
numMthT07               int64
numMthT08               int64
numMthT09               int64
numMthT10               int64
numMthT11               int64
numMthT12               int64
termA                 float64
termB                 float64
termC                 float64
termD                 float64
termE                 float64
termF                 float64
termG     

In [42]:
df.loc[(df['termCount'] == '')] # - no blanks? Not necessarily, see the message below in pink?..
#it means you don't have the results you expect

C:\Users\cinderella\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount


In [43]:
df.loc[(df['termCount'] == -1)] # - no negative numbers

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount
1828,2006-03-22,-1 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.11,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1835,2006-03-23,-2 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.07,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1841,2006-03-24,-3 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.07,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1847,2006-03-27,-6 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.03,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1853,2006-03-28,-7 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,16.00,16.00,15.98,16.00,16.01,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1859,2006-03-29,-8 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,16.00,16.00,16.00,16.00,15.92,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1865,2006-03-30,-9 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,15.86,15.86,15.79,15.79,15.81,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1871,2006-03-31,-10 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,15.76,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1877,2006-04-03,15 days,G (Feb 2007),2007,2,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,15.69,15.69,15.69,15.69,15.74,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1883,2006-04-04,14 days,G (Feb 2007),2007,2,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,15.68,15.68,15.68,15.68,15.64,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1


In [44]:
df.loc[(df['termCount'] == -2)] # - no negative numbers

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount
14889,2012-04-11,6 days,F (Jan 2013),2013,1,2012,2012-04-17,2012-04-18,2011-07-23,2011-09-21,0.0,0.0,0.0,0.0,0.0,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2
14899,2012-04-12,5 days,F (Jan 2013),2013,1,2012,2012-04-17,2012-04-18,2011-07-23,2011-09-21,0.0,0.0,0.0,0.0,0.0,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2
14909,2012-04-13,4 days,F (Jan 2013),2013,1,2012,2012-04-17,2012-04-18,2011-07-23,2011-09-21,0.0,0.0,0.0,0.0,0.0,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2
14919,2012-04-16,1 days,F (Jan 2013),2013,1,2012,2012-04-17,2012-04-18,2011-07-23,2011-09-21,0.0,0.0,0.0,0.0,0.0,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2
14929,2012-04-17,0 days,F (Jan 2013),2013,1,2012,2012-04-17,2012-04-18,2011-07-23,2011-09-21,0.0,0.0,0.0,0.0,0.0,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2


In [45]:
df.loc[(df['termCount'] == 0)] # - no negative numbers

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount
787,2005-01-19,-1 days,F (Jan 2005),2005,1,2005,2005-01-18,2005-01-19,2004-04-24,2004-06-23,0.00,0.00,0.00,0.00,12.772,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
850,2005-02-16,-1 days,G (Feb 2005),2005,2,2005,2005-02-15,2005-02-16,2004-05-22,2004-07-21,0.00,0.00,0.00,0.00,11.293,3,4,5,6,7,8,9,10,11,12,1,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1634,2006-01-18,-1 days,F (Jan 2006),2006,1,2006,2006-01-17,2006-01-18,2005-04-23,2005-06-22,0.00,0.00,0.00,0.00,12.615,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1713,2006-02-15,-1 days,G (Feb 2006),2006,2,2006,2006-02-14,2006-02-15,2005-05-21,2005-07-20,0.00,0.00,0.00,0.00,12.043,3,4,5,6,7,8,9,10,11,12,1,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1768,2006-03-08,13 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,0.000,3,4,5,6,7,8,9,10,11,12,1,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1774,2006-03-09,12 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,15.92,16.08,15.92,16.08,16.040,3,4,5,6,7,8,9,10,11,12,1,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1780,2006-03-10,11 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,16.12,16.18,16.12,16.18,16.130,3,4,5,6,7,8,9,10,11,12,1,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1786,2006-03-13,8 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.130,3,4,5,6,7,8,9,10,11,12,1,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1792,2006-03-14,7 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.110,3,4,5,6,7,8,9,10,11,12,1,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1798,2006-03-15,6 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.090,3,4,5,6,7,8,9,10,11,12,1,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [46]:
df.loc[(df['termCount'] == 13)] # - nothing bigger than december!

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount


Copy the rows with negative numbers and zero termCount to other dbases for further evaluation

In [47]:
df2 = df.loc[(df['termCount'] == -1)].copy()

In [48]:
df3 = df.loc[(df['termCount'] == -2)].copy()

## Figure out  - df4 - what to do with '0' values in termCount

In [49]:
df4 = df.loc[(df['termCount'] == 0)].copy()

In [50]:
df4.shape ## those with zero

(80, 41)

In [51]:
df4.sort_values(['Futures'],inplace=True)

In [52]:
df4.Futures.unique()

array(['F (Jan 2005)', 'F (Jan 2006)', 'F (Jan 2007)', 'F (Jan 2008)',
       'F (Jan 2009)', 'F (Jan 2010)', 'F (Jan 2011)', 'F (Jan 2012)',
       'F (Jan 2013)', 'F (Jan 2014)', 'G (Feb 2005)', 'G (Feb 2006)',
       'G (Feb 2007)', 'G (Feb 2008)', 'G (Feb 2009)', 'G (Feb 2010)',
       'G (Feb 2011)', 'G (Feb 2012)', 'G (Feb 2013)', 'G (Feb 2014)',
       'H (Mar 2006)', 'H (Mar 2007)', 'H (Mar 2008)', 'H (Mar 2009)',
       'H (Mar 2010)', 'H (Mar 2011)', 'H (Mar 2012)', 'H (Mar 2013)',
       'H (Mar 2014)'], dtype=object)

In [53]:
df[df.duplicated()] ##oh well, no duplicate rows in the main df..

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount


In [54]:
df4.diffDates.value_counts()

-1 days     29
12 days      3
1 days       3
7 days       3
4 days       3
11 days      3
6 days       3
13 days      3
0 days       3
8 days       3
5 days       3
15 days      2
-7 days      2
-9 days      2
-8 days      2
-3 days      2
-10 days     2
14 days      2
-6 days      1
18 days      1
-2 days      1
-14 days     1
-15 days     1
19 days      1
-13 days     1
Name: diffDates, dtype: int64

In [55]:
df4

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount
787,2005-01-19,-1 days,F (Jan 2005),2005,1,2005,2005-01-18,2005-01-19,2004-04-24,2004-06-23,0.00,0.00,0.00,0.00,12.772,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1634,2006-01-18,-1 days,F (Jan 2006),2006,1,2006,2006-01-17,2006-01-18,2005-04-23,2005-06-22,0.00,0.00,0.00,0.00,12.615,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3357,2007-01-17,-1 days,F (Jan 2007),2007,1,2007,2007-01-16,2007-01-17,2006-04-22,2006-06-21,0.00,0.00,0.00,0.00,10.706,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5619,2008-01-16,-1 days,F (Jan 2008),2008,1,2008,2008-01-15,2008-01-16,2007-04-21,2007-06-20,0.00,0.00,0.00,0.00,24.180,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8105,2009-01-21,-1 days,F (Jan 2009),2009,1,2009,2009-01-20,2009-01-21,2008-04-26,2008-06-25,0.00,0.00,0.00,0.00,49.880,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
10134,2010-01-20,-1 days,F (Jan 2010),2010,1,2010,2010-01-19,2010-01-20,2009-04-25,2009-06-24,0.00,0.00,0.00,0.00,18.870,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
12224,2011-01-19,-1 days,F (Jan 2011),2011,1,2011,2011-01-18,2011-01-19,2010-04-24,2010-06-23,0.00,0.00,0.00,0.00,16.380,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
14359,2012-01-18,-1 days,F (Jan 2012),2012,1,2012,2012-01-17,2012-01-18,2011-04-23,2011-06-22,0.00,0.00,0.00,0.00,23.640,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
16653,2013-01-16,-1 days,F (Jan 2013),2013,1,2013,2013-01-15,2013-01-16,2012-04-21,2012-06-20,0.00,14.25,14.40,0.00,13.690,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
18918,2014-01-22,-1 days,F (Jan 2014),2014,1,2014,2014-01-21,2014-01-22,2013-04-27,2013-06-26,0.00,0.00,0.00,0.00,12.360,2,3,4,5,6,7,8,9,10,11,12,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


After analyzing the df4 results, we see that rows from the main database should be deleted if termCount == 0 & calYear != futYear. Even though futures can straddle two different months, they should not be doing so when they are the exiting month. In other words, if the futures is the front month, their calendar year should equal their futures year, as in the case of the January listings above. But there is no reason that a trade date of March 20th 2007 should have anything to do with a February futures contract for 2008, unless the futures was to begin it's 9th month, but in the case of a 2008 February futures contract, that would not be until sometime in May of 2007.

In [56]:
df.drop(df.loc[(df['termCount'] == 0) & (df.calYear != df.futYear)].index, inplace=True)

In [57]:
df.shape ## deleted 52 rows with -1 kept 28

(33740, 41)

## Decide what to do with df3 termCount -2 values

In [58]:
df3.shape # those with -2

(5, 41)

In [59]:
df3

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount
14889,2012-04-11,6 days,F (Jan 2013),2013,1,2012,2012-04-17,2012-04-18,2011-07-23,2011-09-21,0.0,0.0,0.0,0.0,0.0,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2
14899,2012-04-12,5 days,F (Jan 2013),2013,1,2012,2012-04-17,2012-04-18,2011-07-23,2011-09-21,0.0,0.0,0.0,0.0,0.0,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2
14909,2012-04-13,4 days,F (Jan 2013),2013,1,2012,2012-04-17,2012-04-18,2011-07-23,2011-09-21,0.0,0.0,0.0,0.0,0.0,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2
14919,2012-04-16,1 days,F (Jan 2013),2013,1,2012,2012-04-17,2012-04-18,2011-07-23,2011-09-21,0.0,0.0,0.0,0.0,0.0,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2
14929,2012-04-17,0 days,F (Jan 2013),2013,1,2012,2012-04-17,2012-04-18,2011-07-23,2011-09-21,0.0,0.0,0.0,0.0,0.0,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2


It appears the Jan 2013 futures were getting ready to assign values to the Futures, however the earliest (9 months) this is valid is around 4/21/2012. Since these rows are all zeros, we will drop them from the main database.

In [60]:
df.drop(df.loc[df['termCount']== -2].index, inplace=True)

In [61]:
df.termCount.value_counts()

 2     4034
 1     4033
 3     3801
 5     3723
 6     3707
 4     3707
 7     3524
 8     3402
 9     2698
 10     530
 12     273
 11     218
-1       57
 0       28
Name: termCount, dtype: int64

## Decide on what to do with df2 termCount -1 values

In [62]:
df2.shape # those with -1

(57, 41)

In [63]:
df2

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount
1828,2006-03-22,-1 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.11,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1835,2006-03-23,-2 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.07,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1841,2006-03-24,-3 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.07,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1847,2006-03-27,-6 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,16.03,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1853,2006-03-28,-7 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,16.00,16.00,15.98,16.00,16.01,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1859,2006-03-29,-8 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,16.00,16.00,16.00,16.00,15.92,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1865,2006-03-30,-9 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,15.86,15.86,15.79,15.79,15.81,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1871,2006-03-31,-10 days,G (Feb 2007),2007,2,2006,2006-03-21,2006-03-22,2005-06-25,2005-08-24,0.00,0.00,0.00,0.00,15.76,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1877,2006-04-03,15 days,G (Feb 2007),2007,2,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,15.69,15.69,15.69,15.69,15.74,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1883,2006-04-04,14 days,G (Feb 2007),2007,2,2006,2006-04-18,2006-04-19,2005-07-23,2005-09-21,15.68,15.68,15.68,15.68,15.64,4,5,6,7,8,9,10,11,12,1,2,3,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1


This is an odd case. All of the values come from February futures for two different years, and all are where the futures year is a year ahead of the calendar year. It *IS* possible for the calendar year to be different from the futures year. The term structures of the VX futures shows 9 periods of data for the VIX, so the earliest say, FebFutures for 2009 could start to record data is 9 months before its expiration (XP is Feb 18 2009 -- 9 months before is around May 24 2008). None of the trade dates are within the 9 month window, which is why they evaluated to -1 termCount. All of these will be deleted from the main database even though they do have values.

In [64]:
df.drop(df.loc[df['termCount']== -1].index, inplace=True)

In [65]:
df.termCount.value_counts()

2     4034
1     4033
3     3801
5     3723
6     3707
4     3707
7     3524
8     3402
9     2698
10     530
12     273
11     218
0       28
Name: termCount, dtype: int64

In [66]:
df.shape

(33678, 41)

In [67]:
df.sort_values(['termCount'], inplace=True) ## if negative numbers, which don't belong, but which
## may need further evaluation

In [68]:
df.tail(20)

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount
4860,2007-09-14,4 days,Q (Aug 2008),2008,8,2007,2007-09-18,2007-09-19,2006-12-23,2007-02-21,0.00,0.00,0.00,0.00,21.360,9,10,11,12,1,2,3,4,5,6,7,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,12
32185,2019-12-16,2 days,X (Nov 2020),2020,11,2019,2019-12-18,2019-12-18,2019-03-23,2019-05-22,0.00,18.65,18.95,0.00,18.675,12,1,2,3,4,5,6,7,8,9,10,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12
4363,2007-06-27,-8 days,M (Jun 2008),2008,6,2007,2007-06-19,2007-06-20,2006-09-23,2006-11-22,16.22,16.22,16.22,16.22,16.150,7,8,9,10,11,12,1,2,3,4,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,12
32197,2019-12-17,1 days,X (Nov 2020),2020,11,2019,2019-12-18,2019-12-18,2019-03-23,2019-05-22,18.99,19.07,18.80,18.90,18.825,12,1,2,3,4,5,6,7,8,9,10,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12
32209,2019-12-18,0 days,X (Nov 2020),2020,11,2019,2019-12-18,2019-12-18,2019-03-23,2019-05-22,18.95,19.15,18.79,18.81,18.750,12,1,2,3,4,5,6,7,8,9,10,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12
2199,2006-06-12,8 days,K (May 2007),2007,5,2006,2006-06-20,2006-06-21,2005-09-24,2005-11-23,16.65,16.70,16.60,16.70,16.710,6,7,8,9,10,11,12,1,2,3,4,5,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,12
4426,2007-07-09,8 days,M (Jun 2008),2008,6,2007,2007-07-17,2007-07-18,2006-10-21,2006-12-20,0.00,0.00,0.00,0.00,16.440,7,8,9,10,11,12,1,2,3,4,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,12
2523,2006-08-16,-1 days,Q (Aug 2007),2007,8,2006,2006-08-15,2006-08-16,2005-11-19,2006-01-18,0.00,0.00,0.00,0.00,16.460,9,10,11,12,1,2,3,4,5,6,7,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,12
10979,2010-06-16,-1 days,M (Jun 2010),2010,6,2010,2010-06-15,2010-06-16,2009-09-19,2009-11-18,0.00,0.00,0.00,0.00,26.110,7,8,9,10,11,12,1,2,3,4,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,12
4307,2007-06-19,0 days,K (May 2008),2008,5,2007,2007-06-19,2007-06-20,2006-09-23,2006-11-22,15.62,15.62,15.62,15.62,15.480,6,7,8,9,10,11,12,1,2,3,4,5,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,12


In [69]:
df.to_csv('df-4.csv')

In [70]:
df.loc[(df['Trade Date'] == '2020-08-10')]

,Trade Date,diffDates,Futures,futYear,futMth,calYear,LTD,t01XP,9MthsStart,7MthsStart,Open,High,Low,Close,Settle,numMthT01,numMthT02,numMthT03,numMthT04,numMthT05,numMthT06,numMthT07,numMthT08,numMthT09,numMthT10,numMthT11,numMthT12,termA,termB,termC,termD,termE,termF,termG,termH,termI,termJ,termK,termL,termM,termCount
33708,2020-08-10,9 days,Q (Aug 2020),2020,8,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,24.60,24.80,23.55,23.85,23.775,8,9,10,11,12,1,2,3,4,5,6,7,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
33709,2020-08-10,9 days,U (Sep 2020),2020,9,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.81,28.01,27.00,27.10,27.125,8,9,10,11,12,1,2,3,4,5,6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2
33710,2020-08-10,9 days,V (Oct 2020),2020,10,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,30.60,30.65,29.90,30.02,30.025,8,9,10,11,12,1,2,3,4,5,6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3
33711,2020-08-10,9 days,X (Nov 2020),2020,11,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,29.00,29.07,28.43,28.60,28.575,8,9,10,11,12,1,2,3,4,5,6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4
33712,2020-08-10,9 days,Z (Dec 2020),2020,12,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.92,28.08,27.45,27.64,27.625,8,9,10,11,12,1,2,3,4,5,6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5
33704,2020-08-10,9 days,F (Jan 2021),2021,1,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,28.40,28.50,27.90,28.09,28.075,8,9,10,11,12,1,2,3,4,5,6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,6
33705,2020-08-10,9 days,G (Feb 2021),2021,2,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.90,28.05,27.52,27.75,27.725,8,9,10,11,12,1,2,3,4,5,6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7
33706,2020-08-10,9 days,H (Mar 2021),2021,3,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,27.60,27.70,27.40,27.46,27.450,8,9,10,11,12,1,2,3,4,5,6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,8
33707,2020-08-10,9 days,J (Apr 2021),2021,4,2020,2020-08-19,2020-08-19,2019-11-23,2020-01-22,0.00,26.05,28.90,0.00,27.475,8,9,10,11,12,1,2,3,4,5,6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,9
